<a href="https://colab.research.google.com/github/ByteAndStored/Software_Architecture_and_Internship_Repo/blob/main/VeriGruplama_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers #hugging face modeli
!pip install torch #pytorch modelin beyni, hesaplama yapıyor
!pip install Pillow #fotoğrafları açıp kullanabilmek için indiririz.Python tek başına yapamaz.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 134.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 753.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
from transformers import pipeline #pipeline kısayoldur --> model yükleme, veriyi verme,sonucu alma
#hugging face modelini yükleriz
classifier = pipeline("image-classification",model = "openai/clip-vit-base-patch32") #hazır modeli çağırıyoruz



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Some weights of CLIPForImageClassification were not initialized from the model checkpoint at openai/clip-vit-base-patch32 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


In [ ]:
import os
input_folder = "/content/drive/MyDrive/images"
output_folder = "/content/drive/MyDrive/Classified/VisDrone_Train_Siniflandirilmis"

In [ ]:
import os
import shutil
from transformers import pipeline

# 1. Modeli yükle (zaten yüklüyse hızlı geçer)
classifier = pipeline("image-classification", model="openai/clip-vit-base-patch32")

# 2. Klasör ayarları
input_folder = "/content/drive/MyDrive/images"
output_folder = "/content/drive/MyDrive/Classified/VisDrone_Train_Siniflandirilmis"  # <-- yeni klasör adı
os.makedirs(output_folder, exist_ok=True)

# 3. Sayacı başlat
counter = 1

# 4. Etiket belirleyen fonksiyon
def classify_labels(labels):
    hava_durumu = ""
    zaman_durumu = ""
    ortam_turu = ""

    # Önce hepsini küçük harfe çevir
    labels = [label.lower() for label in labels]

    # HAVA KOŞULU
    if any(word in labels for word in ["fog", "mist"]):
        hava_durumu = "sisli_"
    elif "snow" in labels or "snowy" in labels:
        hava_durumu = "karli_"
    elif "rain" in labels:
        hava_durumu = "yagmurlu_"
    # Normal ise boş kalır (yani hava koşulu etiketi olmaz)

    # ZAMAN / IŞIK DURUMU
    if "night" in labels:
        zaman_durumu = "gece_"
    elif "daytime" in labels or "sunny" in labels or "cloudy" in labels:
        zaman_durumu = "gunduz_"
    elif "thermal" in labels:
        zaman_durumu = "termal_"
    else:
        # Belirgin ışık bilgisi yoksa gece sayıyoruz (senin kuralına göre)
        zaman_durumu = "gece_"

    # ORTAM TÜRÜ
    if any(word in labels for word in ["building", "road", "car", "street"]):
        ortam_turu = "sehir"
    elif any(word in labels for word in ["forest", "trees", "jungle"]):
        ortam_turu = "orman"
    elif any(word in labels for word in ["sea", "ocean", "ship", "boat"]):
        ortam_turu = "deniz"
    elif any(word in labels for word in ["field", "open area", "plain", "meadow"]):
        ortam_turu = "acik_alan"
    else:
        ortam_turu = "belirsiz"  # hiçbir ortam bulunamazsa

    return hava_durumu + zaman_durumu + ortam_turu

# 5. Fotoğrafları sırayla işle
for file_name in sorted(os.listdir(input_folder)):
    file_path = os.path.join(input_folder, file_name)
    if not file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue

    try:
        # Fotoğrafı modelle tahmin et
        result = classifier(file_path)
        labels = [r['label'].lower() for r in result]

        # Ortam ismini belirle
        environment_name = classify_labels(labels)

        # Yeni dosya adı oluştur
        new_file_name = f"{environment_name} {counter:07d}.jpg"

        # Fotoğrafı yeni klasöre kopyala
        shutil.copy(file_path, os.path.join(output_folder, new_file_name))

        # Sayacı arttır
        counter += 1

        print(f"{file_name} -> {new_file_name}")  # ilerleme çıktısı
        print(f"İşleniyor: {file_name}")
        result = classifier(file_path)
        labels = [r['label'].lower() for r in result]
        print(f"Tahmin Edilen Etiketler: {labels}")

    except Exception as e:
        print(f"Hata oluştu {file_name} dosyasında: {e}")


Some weights of CLIPForImageClassification were not initialized from the model checkpoint at openai/clip-vit-base-patch32 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


0000002_00005_d_0000014.jpg -> gece_belirsiz 0000001.jpg
İşleniyor: 0000002_00005_d_0000014.jpg
Tahmin Edilen Etiketler: ['label_0', 'label_1']
0000002_00448_d_0000015.jpg -> gece_belirsiz 0000002.jpg
İşleniyor: 0000002_00448_d_0000015.jpg
Tahmin Edilen Etiketler: ['label_0', 'label_1']
0000003_00231_d_0000016.jpg -> gece_belirsiz 0000003.jpg
İşleniyor: 0000003_00231_d_0000016.jpg
Tahmin Edilen Etiketler: ['label_0', 'label_1']
0000007_04999_d_0000036.jpg -> gece_belirsiz 0000004.jpg
İşleniyor: 0000007_04999_d_0000036.jpg
Tahmin Edilen Etiketler: ['label_0', 'label_1']


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Görüntülenen çıkış son 5000 satıra kısaltıldı.
9999966_00000_d_0000054.jpg -> gece_belirsiz 0004020.jpg
İşleniyor: 9999966_00000_d_0000054.jpg
Tahmin Edilen Etiketler: ['label_0', 'label_1']
9999966_00000_d_0000055.jpg -> gece_belirsiz 0004021.jpg
İşleniyor: 9999966_00000_d_0000055.jpg
Tahmin Edilen Etiketler: ['label_0', 'label_1']
9999966_00000_d_0000056.jpg -> gece_belirsiz 0004022.jpg
İşleniyor: 9999966_00000_d_0000056.jpg
Tahmin Edilen Etiketler: ['label_0', 'label_1']
9999966_00000_d_0000057.jpg -> gece_belirsiz 0004023.jpg
İşleniyor: 9999966_00000_d_0000057.jpg
Tahmin Edilen Etiketler: ['label_0', 'label_1']
9999966_00000_d_0000058.jpg -> gece_belirsiz 0004024.jpg
İşleniyor: 9999966_00000_d_0000058.jpg
Tahmin Edilen Etiketler: ['label_0', 'label_1']
9999966_00000_d_0000059.jpg -> gece_belirsiz 0004025.jpg
İşleniyor: 9999966_00000_d_0000059.jpg
Tahmin Edilen Etiketler: ['label_0', 'label_1']
9999966_00000_d_0000060.jpg -> gece_belirsiz 0004026.jpg
İşleniyor: 9999966_00000_d_00000